## Load Module

In [1]:
from gate_util import *
from merge_func import *
from commute_func import *
from autocomm import *

## MCTR

In [2]:
def multi_controlled(num_qubit, qb_per_node):
    gates = []
    # cq: 0-48 tq: 98 # cq: 49-98 tq: 99
    cfg = [([i for i in range(num_qubit//2-1)],num_qubit-2),([i for i in range(num_qubit//2-1, num_qubit-1)],num_qubit-1),([i for i in range(num_qubit//2-1)],num_qubit-2),([i for i in range(num_qubit//2-1, num_qubit-1)],num_qubit-1)]
    for cq, tq in cfg:
        idle_qubit = []
        for i in range(num_qubit):
            if i not in cq and i != tq:
                idle_qubit.append(i)
        rev_gate = []
        toffoli_tq = tq
        for i, idx in enumerate(cq[2:][::-1]):
            # cq: idx, idle_qubit
            toffoli_cq1 = idx
            toffoli_cq2 = idle_qubit[-i-1]
            gates.append(build_toffoli_gate(toffoli_cq1,toffoli_cq2,toffoli_tq))
            rev_gate.insert(0, build_toffoli_gate(toffoli_cq1,toffoli_cq2,toffoli_tq))
            toffoli_tq = toffoli_cq2
        toffoli_cq1 = cq[0]
        toffoli_cq2 = cq[1]
        toffoli_tq = idle_qubit[-i-1]
        gates.append(build_toffoli_gate(toffoli_cq1,toffoli_cq2,toffoli_tq))
        for gl in rev_gate:
            gates.append(gl)
        rev_gate = []
        for i, idx in enumerate(cq[2:-1][::-1]):
            # cq: idx, idle_qubit
            toffoli_cq1 = idx
            toffoli_cq2 = idle_qubit[-i-1]
            gates.append(build_toffoli_gate(toffoli_cq1,toffoli_cq2,toffoli_tq))
            rev_gate.insert(0, build_toffoli_gate(toffoli_cq1,toffoli_cq2,toffoli_tq))
            toffoli_tq = toffoli_cq2
        toffoli_cq1 = cq[0]
        toffoli_cq2 = cq[1]
        toffoli_tq = idle_qubit[-i-1]
        gates.append(build_toffoli_gate(toffoli_cq1,toffoli_cq2,toffoli_tq))
        for gl in rev_gate:
            gates.append(gl)
    qubit_node_mapping = [i//qb_per_node for i in range(num_qubit)]
    return gates, qubit_node_mapping

In [3]:
num_q, qb_per_node = 100, 10
gate_list, qubit_node_mapping = multi_controlled(num_q, qb_per_node)
epr_cnt, all_latency, assigned_gate_block_list1 = autocomm_full(gate_list, qubit_node_mapping, allow_local_view=True, aggregate_iter_cnt=6, schedule_iter_cnt=10)
print(epr_cnt, all_latency)

cnt = 0
for sub_gl in gate_list:
    for g in sub_gl:
        qbs = gate_qubits(g)
        if len(qbs) == 2:
            if qubit_node_mapping[qbs[0]] != qubit_node_mapping[qbs[1]]:
                cnt += 1
print(cnt, epr_cnt, cnt/epr_cnt)

1072 14270.70000000052
3192 1072 2.9776119402985075


In [4]:
num_q, qb_per_node = 100, 10
gate_list, qubit_node_mapping = multi_controlled(num_q, qb_per_node)
new_gate_list = []
for sub_gl in gate_list:
    new_gate_list.extend(sub_gl)

epr_cnt, all_latency, assigned_gate_block_list1 = autocomm_full(new_gate_list, qubit_node_mapping, allow_local_view=False, allow_test_merge=True, aggregate_iter_cnt=6, schedule_iter_cnt=10)
print(epr_cnt, all_latency)

1072 14270.70000000052


## BV

In [5]:
def BV(num_qubits, qb_per_node):
    gate_list = []
    for i in range(num_qubits-1):
        gate_list.append(build_gate("CX", [0, i+1]))
    qubit_node_mapping = [i//qb_per_node for i in range(num_qubits)] # optimal mapping obtained
    return gate_list, qubit_node_mapping

In [9]:
num_q, qb_per_node = 100, 10
gate_list, qubit_node_mapping = BV(num_q, qb_per_node)
epr_cnt, all_latency, assigned_gate_block_list1 = autocomm_full(gate_list, qubit_node_mapping, aggregate_iter_cnt=num_q//qb_per_node, schedule_iter_cnt=num_q//qb_per_node)
print(epr_cnt, all_latency)

9 221.6999999999999


## QFT

In [10]:
def QFT(num_qubits, qb_per_node):
    gate_list = []
    for i in range(num_qubits-1):
        gate_list.append(build_H_gate(i))
        for j in range(i+1, num_qubits):
            gate_list.append(build_CX_gate(j,i))
            gate_list.append(build_RZ_gate(i,angle=-np.pi/4/2**(j-i)))
            gate_list.append(build_CX_gate(j,i))
            gate_list.append(build_RZ_gate(i,angle=np.pi/4/2**(j-i)))
    qubit_node_mapping = [i//qb_per_node for i in range(num_qubits)] # optimal mapping obtained
    return gate_list, qubit_node_mapping

In [13]:
num_q, qb_per_node = 300, 10
gate_list, qubit_node_mapping = QFT(num_q, qb_per_node)
epr_cnt, all_latency, assigned_gate_block_list1 = autocomm_full(gate_list, qubit_node_mapping, allow_gate_pattern=True, aggregate_iter_cnt=1, schedule_iter_cnt=num_q//qb_per_node)
print(epr_cnt, all_latency)

4640 43644.69999999408


## QAOA

In [14]:
def QAOA(num_terms, num_qubits, qb_per_node):
    import random
    gate_list = []
    for i in range(num_terms):
        qa, qb = random.sample(list(range(num_qubits)), 2)
        gate_list.append(build_CX_gate(qa,qb))
        gate_list.append(build_RZ_gate(qb,angle=0.1))
        gate_list.append(build_CX_gate(qa,qb))
    qubit_node_mapping = [i//qb_per_node for i in range(num_qubits)]
    return gate_list, qubit_node_mapping

In [16]:
num_q, qb_per_node = 100, 10
gate_list, qubit_node_mapping = QAOA(100, num_q, qb_per_node)
epr_cnt, all_latency, assigned_gate_block_list1 = autocomm_full(gate_list, qubit_node_mapping, allow_gate_pattern=True, aggregate_iter_cnt=num_q//qb_per_node, schedule_iter_cnt=num_q//qb_per_node)
cnt = 0
for g in gate_list:
    qbs = gate_qubits(g)
    if len(qbs) == 2:
        if qubit_node_mapping[qbs[0]] != qubit_node_mapping[qbs[1]]:
            cnt += 1
print(cnt/epr_cnt)

2.1176470588235294


## RCA

In [17]:
def RCA(num_qubits, qb_per_node):
    start_qb = 0
    gate_list = []
    while start_qb < num_qubits-3:
        qa, qb, qc = start_qb, start_qb+1, start_qb+2
        gate_list.append(build_CX_gate(qc,qb))
        gate_list.append(build_CX_gate(qc,qa))
        gate_list += build_toffoli_gate(qa, qb, qc)
        start_qb += 2
    start_qb -= 2
    gate_list.append(build_CX_gate(start_qb,start_qb+1))
    while start_qb > 0:
        qa, qb, qc = start_qb-2,start_qb-1,start_qb
        gate_list += build_toffoli_gate(qa, qb, qc)
        gate_list.append(build_CX_gate(qc,qa))
        gate_list.append(build_CX_gate(qa,qb))
        start_qb -= 2
    qubit_node_mapping = [i//qb_per_node for i in range(num_qubits)] # the optimal one
    return gate_list, qubit_node_mapping

In [23]:
num_q, qb_per_node = 100, 10
gate_list, qubit_node_mapping = RCA(num_q, qb_per_node)
epr_cnt, all_latency, assigned_gate_block_list1 = autocomm_full(gate_list, qubit_node_mapping, aggregate_iter_cnt=3, schedule_iter_cnt=num_q//qb_per_node)
print(epr_cnt, all_latency)

36 822.1000000000079
